<a href="https://colab.research.google.com/github/mHash1m/My-Python-Experience/blob/master/SocialDistancingDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

My first Project on OpenCV.

This notebook is inspired by this [guide](https://www.pyimagesearch.com/2020/06/01/opencv-social-distancing-detector/).

In [1]:
#import required packages
import numpy as np
import cv2
from scipy.spatial import distance as dist
import argparse
import imutils
import os 

In [2]:
#Our Base Path in colab
PATH = "drive/My Drive/Social Distancing Detection"

#input and output files
output = os.path.sep.join([PATH, "output.avi"])
input = os.path.sep.join([PATH, "videoplayback"])

# # load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([PATH, "coco-names"])
LABELS = open(labelsPath).read().strip().split("\n")

# # derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([PATH, "yolov3.weights"])
configPath = os.path.sep.join([PATH, "yolo.cfg"])

In [3]:
#Initialize Variables for configuration

# initialize minimum probability to filter weak detections along with
# the threshold when applying non-maxima suppression
MIN_CONF = 0.3
NMS_THRESH = 0.3

# # boolean indicating if NVIDIA CUDA GPU should be used. Kept this False as I didnt wanna get into setting up the GPU requirements on colab.
# USE_GPU = False

#Min Distance to be observed while social distancing in pixels. Anything above this would be marked as a violation.
MIN_DISTANCE = 40

display = 0

Alrighty. We are all set with the variables. Compiling them in a single cell helps experimentation during the rest of the project. Now to get started with our function to use the Yolo model to detect people.

In [4]:
def detect_people(frame, model, outputL_names, person_idx = 0):
  (H, W) = frame.shape[:2]
  results = []

  boxes = []
  centroids = []
  confidences = []
  #Creating a blob and performing forwards pass.
  blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
  model.setInput(blob)
  layerOutputs = model.forward(outputL_names)
  for output in layerOutputs:
    for detection in output:
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]
      if classID == person_idx and confidence > MIN_CONF:
        #scale our bounding box according to the frame size.
        box = detection[0:4] * np.array([W, H, W, H])
        #derive top-left corner
        (cX, cY, width, height) = box.astype("int")
        blX = int(cX - (width/2))
        blY = int(cY - (height/2))

        #update our lists
        boxes.append([blX, blY, int(width), int(height)])
        centroids.append((cX, cY))
        confidences.append(float(confidence))

  #now lets apply non-maxima suppression
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)
  if len(idxs) > 0:
    for idx in idxs.flatten():
      (x, y) = (boxes[idx][0], boxes[idx][1])
      (w, h) = (boxes[idx][2], boxes[idx][3])
      results.append((confidences[idx], (x, y, x+w, y+h), centroids[idx]))
  return results

So, we're all set for detecting people and building bounding boxes around the detected persons. All thats left is to use the controids of the detected people and calculate the distance between them(in pixels of course). 

If the distance exceeds our decided minimum distance to be observed, we mark it as a violation.

In [5]:
#Load our YOLO MODEL trained on COCO dataset from the drive.
print("Loading our YOLO model ....")
model = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

Loading our YOLO model ....


In [6]:
#Determine the only output layer names we need from the YOLO model.
outputL_names = model.getLayerNames()
outputL_names = [outputL_names[i[0] - 1] for i in model.getUnconnectedOutLayers()]

#initialize our video stream and pointer to output file.
print("Accessing the Video Stream ....")
vs = cv2.VideoCapture(input if input else 0)
writer = None

Accessing the Video Stream ....


In [7]:
#Lets begin reading processing the video frames.

while True:
  (grabbed, frame) = vs.read()

  #break if no frames grabbed
  if not grabbed:
    break
  #scale
  frame = imutils.resize(frame, width = 700)

  #lets detect some people!
  results = detect_people(frame, model, outputL_names, person_idx=LABELS.index("person"))

  #We will keep out Violations in a set
  violations = set()
  v_ = [[] for _ in range(len(results))]
  #Start calculating pairwise distances of peple.
  if len(results) >= 2:
    centroids = np.array([r[2] for r in results])
    #calculate Euclidean Distances between pairs of centroids
    D = dist.cdist(centroids, centroids, metric="euclidean")
    for i in range(0, D.shape[0]):
      for j in range(i+1, D.shape[1]):
        #check if the distance raises a violation
        if D[i, j] < MIN_DISTANCE:
          #Add the indexes to our set of violations
          violations.add(i)
          violations.add(j)
          v_[i].append(j)
    for (i , (conf, bbox, centroid)) in enumerate(results):
      (startX, startY, endX, endY) = bbox
      (cX, cY) = centroid
      #set the default color of bounding boxes to green.
      color = (0 , 255, 0)
      #set the violation color of bounding boxes to red.
      if i in violations:
        color = (0, 0, 255)
      #draw bboxes, centroids and lines b/w centroids
      cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
      cv2.circle(frame, (cX, cY), 5, color, 1)
      for v in v_[i]:
        (_, _, (c2X, c2Y)) = results[v]
        cv2.line(frame, (cX, cY), (c2X, c2Y), color, 1)
  text = "Social Distancing Violations: {}".format(len(violations))
  cv2.putText(frame, text, (10, frame.shape[0] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255) if (len(violations) > 0) else (0, 255, 0), 3)

  # check to see if the output frame should be displayed to our
  # screen
  if display > 0:
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
      break
  # if an output video file path has been supplied and the video
  # writer has not been initialized, do so now
  if output != "" and writer is None:
    # initialize our video writer
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter(output, fourcc, 25,
      (frame.shape[1], frame.shape[0]), True)
  # if the video writer is not None, write the frame to the output
  # video file
  if writer is not None:
    writer.write(frame)